In [1]:
import json

from jury import Jury

In [2]:
mt_predictions = [
    ["the cat is on the mat", "There is cat playing on the mat"], 
    ["Look! a wonderful day."]
]
mt_references = [
    ["the cat is playing on the mat.", "The cat plays on the mat."],
    ["Today is a wonderful day", "The weather outside is wonderful."],
]

MT_METRICS = [
    "bleu", 
    "meteor",
    "rouge", 
    "sacrebleu", 
#     "bertscore"
]

In [ ]:
mt_jury = Jury(metrics=MT_METRICS, run_concurrent=True)
scores = mt_jury.evaluate(predictions=mt_predictions, references=mt_references)
print(json.dumps(scores, indent=4))

  0%|                                                     | 0/2 [00:00<?, ?it/s]

In [ ]:
from jury.metrics.squad import SQUAD

class SQUAD_EM(SQUAD):
    def __init__(self):
        metric_name = "squad" 
        resulting_name = "SQUAD-EM"
        super().__init__(metric_name=metric_name, resulting_name=resulting_name)
        
    def _postprocess(self, result, return_dict):
        result = {self.metric_name: result["exact_match"] / 100}
        return result

In [ ]:
qa_predictions = ["1917", "Albert Einstein", "foo bar"]
qa_references = ["1917", "Einstein", "foo bar foobar"]

QA_METRICS = [
    SQUAD_EM(),
    SQUAD()
]

In [ ]:
qa_jury = Jury(metrics=QA_METRICS, run_concurrent=False)
scores = qa_jury.evaluate(predictions=qa_predictions, references=qa_references)
print(json.dumps(scores, indent=4))